In [24]:
import rospy,tf, numpy as np, transformations, json, rospkg, yaml
from os import system
from scipy.optimize import minimize
import transformations
import time

In [2]:
rospy.init_node('calibrator')

In [3]:
rospy.set_param('/optitrack/objects', ['/robot/calibrator'])

In [4]:
tfl = tf.TransformListener(True, rospy.Duration(2))  # tf will have 2 seconds of cache

In [5]:
rospack = rospkg.RosPack()

In [6]:
world_frame_robot = rospy.get_param('/optitrack/world_frame') # The world frame of the robot
world_frame_opt = 'optitrack_frame' # The world frame of optitrack
calib_frame_robot = 'right_gripper' # The frame used for calibration in the robot's frame
calib_frame_opt = '/robot/calibrator' # The frame used for calibration in the optitrack's frame

In [7]:
print "Now run the publisher: roslaunch optitrack_publisher optitrack_publisher.launch ip:=<IP> world:={}".format(world_frame_robot)

Now run the publisher: roslaunch optitrack_publisher optitrack_publisher.launch ip:=<IP> world:=base


Also check that the ROS master and the local machine are up-to-date regarding time, to synchronize your local machine's time execute:
```
ntpdate -q ntp.ubuntu.com
```

## Definition of functions
Nothing to do here, just executing...
### Record calibration points

In [8]:
def record_calibration_points(continuous = True, duration=60, min_dist=0.01, max_dur=0.05):
    mat_robot = [] # Matrix of all calibration points of calib_frame_robot in frame world_frame_robot (position only)
    mat_opt = [] # Matrix of all calibration points of calib_frame_opt in frame world_frame_opt (position only)
    max_dur = rospy.Duration(max_dur) # seconds
    duration = rospy.Duration(duration)
    
    start = rospy.Time.now()
    last_point = None
    entry = ""
    while continuous and rospy.Time.now()<start+duration or not continuous and entry=="":   
        ref_time = tfl.getLatestCommonTime(world_frame_robot, calib_frame_opt)
        now = rospy.Time.now()
        
        if ref_time > now - max_dur:
            try:
                pose_rg_robot = tfl.lookupTransform(world_frame_robot, calib_frame_robot, rospy.Time(0))
            except Exception, e:
                print "Robot <-> Optitrack transformation not available at the last known common time:", e.message
            else:
                if last_point is None or transformations.distance(pose_rg_robot, last_point)>min_dist:
                    try:
                        pose_rg_opt = tfl.lookupTransform(world_frame_opt, calib_frame_opt, rospy.Time(0))
                    except:
                        print "Optitrack Marker not visible at the last know common time"
                    else:
                        mat_robot.append(np.array(pose_rg_robot))
                        mat_opt.append(np.array(pose_rg_opt))
                        last_point = pose_rg_robot
                        system('beep')
        else:
            print "TFs are", (now - ref_time).to_sec(), "sec late"
        
        if continuous:
            rospy.sleep(0.25)
        else:
            entry = raw_input("Press enter to record a new point or q-enter to quit ({} points)".format(len(mat_robot)))
    return mat_opt, mat_robot

### Formating functions

In [9]:
def extract_transforms(flat_transforms):
    # a transform is 3 pos and 4 rot
    nb_transform = len(flat_transforms) / 7
    list_transforms = []
    for i in range(nb_transform):
        pose = []
        # extract the pose
        pose.append(flat_transforms[i * 7:i * 7 + 3])
        pose.append(flat_transforms[i * 7 + 3:i * 7 + 7])
        # append it to the list of transforms
        list_transforms.append(pose)
    return list_transforms
        
def result_to_calibration_matrix(result):
    calibration_matrix = transformations.inverse_transform(result)
    return [map(float, calibration_matrix[0]), map(float, calibration_matrix[1].tolist())]

### Cost function for the optimization

In [62]:
def evaluate_calibration(calibrations, coords_robot, coords_opt):
    def distance_cost(pose1, pose2, rot_coeff=2):
        pos_cost = 0
        # calculate position ditance
        pos_cost = np.linalg.norm(np.array(pose1[0]) - np.array(pose2[0]))
        # distance between two quaternions
        rot_cost = 1 - np.inner(pose1[1], pose2[1])**2
        return pos_cost + rot_coeff * rot_cost

    # first extract the transformations
    list_calibr = extract_transforms(calibrations)
    # set the base transform
    A = list_calibr[0]
    B = list_calibr[1]
    # loop trough all the transforms
    cost = 0
    for i in range(len(coords_robot)):
        robot = coords_robot[i]
        opt = coords_opt[i]
        product = transformations.multiply_transform(robot, B)
        product = transformations.multiply_transform(A, product)
        
        cost += distance_cost(opt, product)
    return cost

In [63]:
cons = ({'type': 'eq', 'fun': lambda x:  abs(np.linalg.norm(x[3:7])- 1)},
        {'type': 'eq', 'fun': lambda x:  abs(np.linalg.norm(x[10:14])- 1)})

### Functions for quality checking

In [36]:
def calculate_position_error(A, B, coords_robot, coords_opt):
    norm = 0.
    # precision error
    for i in range(len(coords_robot)):
        robot = coords_robot[i]
        opt = coords_opt[i]
        product = tranformations.multiply_tranform(robot, B)
        product = tranformations.multiply_tranform(A, product)
        norm += np.linalg.norm(opt[0], product[0])
    return norm

## Acquisition of points for calibration by moving the arm...

In [12]:
# Record during 60 sec... set continuous=False for an interactive mode
mat_opt, mat_robot = record_calibration_points(continuous=True)

In [13]:
print len(mat_opt), "points recorded"

129 points recorded


## Optimization of the calibration matrix

In [60]:
initial_guess = [0,0,0,0,0,0,1]*2

In [64]:
t0 = time.time()
# Be patient, this cell can be long to execute...
result = minimize(evaluate_calibration, initial_guess, args=(mat_robot, mat_opt, ),
                  method='SLSQP', constraints=cons)
print time.time()-t0, "seconds of optimization"

KeyboardInterrupt: 

In [46]:
print result

     fun: 0.53784406439885435
     jac: array([-0.01361901, -0.00231588, -0.00189607,  0.00195947, -0.01379845,
       -0.00607565,  0.00055345,  0.0068095 ,  0.01071879, -0.00121898,
       -0.00215551, -0.0045734 ,  0.0026232 , -0.00164585,  0.        ])
 message: 'Optimization terminated successfully.'
    nfev: 804
     nit: 46
    njev: 46
  status: 0
 success: True
       x: array([-0.53879008, -0.18859231,  0.91843086, -0.00234543,  0.01890137,
        0.04220829,  0.99892727, -0.01749826, -0.00993011, -0.02432433,
        0.35919949, -0.58972421, -0.3289611 ,  0.64419383])


In [48]:
result_list = extract_transforms(result.x)

## Dumping A i.e. the calibration matrix (the transformation between the optitrack frame and the robot base frame)

In [49]:
calibration_matrix_a = result_to_calibration_matrix(result_list[0])

In [50]:
rospy.set_param("/optitrack/calibration_matrix", calibration_matrix_a)

In [51]:
with open(rospack.get_path("optitrack_publisher")+"/config/calibration_matrix.yaml", 'w') as f:
    yaml.dump(calibration_matrix_a, f)

## Dumping B (the transformation between the optitrack marker and the end effector frame)

In [52]:
calibration_matrix_b = result_to_calibration_matrix(result_list[1])

In [53]:
with open(rospack.get_path("optitrack_publisher")+"/config/calibration_matrix_b.yaml", 'w') as f:
    yaml.dump(calibration_matrix_b, f)

## Testing calibration quality

In [ ]:
mat_opt_check, mat_robot_check = record_calibration_points(False)

In [ ]:
print len(mat_opt), "points recorded"

In [ ]:
calculate_cost(result_a, result_b, mat_robot_check, mat_opt_check)

In [ ]:
avg_error = calculate_position_error(result_a, result_b, mat_robot_check, mat_opt_check)/len(mat_opt_check)

In [ ]:
avg_error